In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import matplotlib.pyplot as plt
from scipy.stats import kendalltau

# Load and preprocess data
df_eth = pd.read_csv('./orderbook_data/eth_cbs_mcd.csv')

# Group data by minute and calculate the last value for best bid price and quantity
df = df_eth.groupby('by_minute')[['best_bid', 'best_ask', 'premium', 'rolling_premium', 'bid_usd']].last().reset_index()
df['by_minute'] = pd.to_datetime(df['by_minute']).dt.strftime('%Y/%m/%d %H:%M:%S')

df = df.set_index('by_minute').dropna()

# Calculate features
df['ema_5min'] = df['best_bid'].ewm(span=5, adjust=False).mean()
df['ma_5min'] = df['best_bid'].rolling(window=5, min_periods=1).mean()
df['spread'] = (df['best_ask']-df['best_bid'])/df['best_ask']

# Calculate Kendall's Tau with a 10-minute window
def rolling_kendall_tau(series, window=10):
    tau_values = []
    for i in range(len(series)):
        if i < window - 1:
            tau_values.append(0)  # Fill initial values with 0
        else:
            window_data = series[i-window+1:i+1]
            time_idx = np.arange(window)
            tau, _ = kendalltau(time_idx, window_data)
            tau_values.append(tau if not np.isnan(tau) else 0)
    return tau_values

df['kendall_tau'] = rolling_kendall_tau(df['best_bid'])
df['kendall_tau_diff'] = df['kendall_tau'].diff().fillna(0)

# Split data for training and testing
split_time = pd.Timestamp('2024-11-02 03:54:00', tz='UTC')
split_time_rounded = split_time.floor('1min').strftime('%Y/%m/%d %H:%M:%S')

train_data = df[df.index <= split_time_rounded]
test_data = df[df.index > split_time_rounded].head(3)

print(f"\nTraining data ends at: {train_data.index[-1]}")
print(f"Predicting for: {test_data.index[0]}")

# Prepare data for ARIMA with exogenous variables
y_train = train_data['ema_5min']
train_exog = train_data[['best_ask', 'premium']]
test_exog = test_data[['best_ask', 'premium']]

# Plot ACF & PACF of the differenced moving average
plt.figure(figsize=(8, 4))
plot_acf(y_train.diff(periods=5).dropna(), lags=10, ax=plt.gca())
plt.title('ACF of 5-Minute Moving Average (Differenced)')

plt.figure(figsize=(8, 4))
plot_pacf(y_train.diff(periods=5).dropna(), lags=10, ax=plt.gca())
plt.title('PACF of 5-Minute Moving Average (Differenced)')
plt.show()

# Fit ARIMA model with exogenous variable
arima_model_exog = sm.tsa.ARIMA(
    y_train, 
    order=(4, 1, 4), 
    exog=train_exog
)
arima_result_exog = arima_model_exog.fit()

# Residual diagnostics
residuals_exog = arima_result_exog.resid

# Correctly provide 3 rows of test exogenous data
forecast_exog = arima_result_exog.get_forecast(steps=3, exog=test_exog.iloc[:3])
predicted_values_exog = forecast_exog.predicted_mean
conf_int_exog = forecast_exog.conf_int()

# Extract actual values for comparison
actual_values = test_data['ema_5min'].iloc[:3]

# Calculate errors for each step
absolute_errors = abs(predicted_values_exog.values - actual_values.values)
percentage_errors = (absolute_errors / actual_values) * 100

# Print Results
print("\n3-Step Prediction Results with Exogenous Variable:")
for step in range(3):
    print(f"Step {step + 1}:")
    print(f"  Actual Value: {actual_values.iloc[step]:.2f}")
    print(f"  Predicted Value: {predicted_values_exog.iloc[step]:.2f}")
    print(f"  Absolute Error: {absolute_errors[step]:.2f}")
    print(f"  Percentage Error: {percentage_errors[step]:.2f}%")
    print(f"  Confidence Interval (95%): [{conf_int_exog.iloc[step, 0]:.2f}, {conf_int_exog.iloc[step, 1]:.2f}]\n")

# Plot last 30 minutes of training data and predictions
last_n = 30
last_train = train_data[-last_n:]
fitted_values_exog = arima_result_exog.get_prediction(start=last_train.index[0], exog=train_exog[-last_n:])

fig, ax1 = plt.subplots(figsize=(20, 6))

# Plot actual training data
ax1.plot(last_train.index, last_train['ema_5min'], label='Actual (Train)', color='blue')

# Plot fitted values on training data
ax1.plot(last_train.index, fitted_values_exog.predicted_mean, label='Fitted', color='green', linestyle='--')

# Plot actual and predicted values for the test set
ax1.plot([last_train.index[-1]] + list(test_data.index[:3]), 
         [last_train['ema_5min'].iloc[-1]] + list(actual_values), 
         label='Actual (Test)', color='blue', linestyle='--')

ax1.plot([last_train.index[-1]] + list(test_data.index[:3]), 
         [last_train['ema_5min'].iloc[-1]] + list(predicted_values_exog), 
         label='Forecast', color='red', linestyle='--')

# Confidence intervals
for step in range(3):
    ax1.fill_between([test_data.index[step]], 
                     conf_int_exog.iloc[step, 0], 
                     conf_int_exog.iloc[step, 1], 
                     color='pink', alpha=0.3)

ax1.set_title('Last 30 Minutes of Training Data with 3-Step Forecast (Exogenous)')
ax1.set_xlabel('Time')
ax1.set_ylabel('5-Minute Moving Average Price', color='blue')
ax1.tick_params(axis='y', labelcolor='blue')
ax1.legend(loc='upper left')
plt.xticks(rotation=90)

ax2 = ax1.twinx()
ax2.plot(last_train.index, last_train['bid_usd'], label='Coinbase Best Bid', color='orange', linestyle=':')
ax2.set_ylabel('Premium', color='orange')
ax2.tick_params(axis='y', labelcolor='orange')
ax2.legend(loc='upper right')

plt.tight_layout()
plt.show()